#Import libraies and packages

In [ ]:
pip install ray torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 39.7 MB/s 
     |████████████████████████████████| 468 kB 43.7 MB/s 


In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  if sys.path[0] == '':


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model

In [ ]:
import tensorflow as tf

def ConvNet(config, len_classes=2):
    input = tf.keras.layers.Input(shape=(43893, 1))
    x = input
    x = tf.keras.layers.Conv1D(filters=config['conv_block1_filters'], kernel_size=(8), strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if config['fc_layer_type'] == 'dense':
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)

        # Fully connected layer 1
        x = tf.keras.layers.Dense(units=config['fc1_units'])(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        # Fully connected layer 2
        x = tf.keras.layers.Dense(units=len_classes)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        predictions = tf.keras.layers.Dense(3,tf.keras.layers.Activation('softmax'))(x)

    else:
        # Fully connected layer 1
        x = tf.keras.layers.Conv1D(filters=config['fc1_units'], kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # Fully connected layer 2
        x = tf.keras.layers.Conv1D(filters=len_classes, kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        predictions = tf.keras.layers.Dense(3,tf.keras.layers.Activation('softmax'))(x)
    print(predictions)
    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model
   

## Metrics

In [ ]:
from keras import backend as K
import random
from sklearn.metrics import f1_score, precision_score, recall_score


def recall_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    recall = recall_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return recall

def precision_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    precision = precision_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return precision

def f1_m(y_true, y_pred):
    y_true.numpy()
    y_pred.numpy()
    f1 = f1_score(y_true, np.argmax(y_pred, axis = 1), average='weighted', zero_division = 1)
    return f1

## Training function

In [ ]:
from sklearn import preprocessing

def train_mnist(config):
  path ='/content/drive/MyDrive/ART_Inv/CNN/Ray_Tune/Clinical_data_and_RNA_total_Features_PFS.csv'
  data_frame = pd.read_csv(path)
  
  from sklearn.model_selection import train_test_split
  X = data_frame.iloc[:,28:43921  ]   
  Y=[]
  for i in range (len(data_frame)):
      if data_frame.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
          Y.append(0)
      elif data_frame.PFS[i]<6:
          Y.append(1)
      else:
          Y.append(2)# If PFS is over 3 months, I will consider it as Responder (R)
  scaler = preprocessing.MinMaxScaler()
  names = X.columns
  d = scaler.fit_transform(X)
  X = pd.DataFrame(d, columns=names)
  XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, stratify = Y)
  # Convert sets to arrays
  XTrain = XTrain.values
  XTest = XTest.values
  # It is mandatory to transform Y list into array for trainning the model
  yTrain=np.array(yTrain)
  yTest=np.array(yTest)

  X_train = XTrain.reshape(XTrain.shape[0], 43893 , 1)
  X_test = XTest.reshape(XTest.shape[0], 43893, 1)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  # Create model
  model = ConvNet(config)
  # Compile model with losses and metrics
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =config['lr']),
                # tf.keras.optimizers.RMSprop(learning_rate =config['lr']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy', f1_m, precision_m, recall_m], run_eagerly=True)
  # Start model training
  history_m = model.fit(X_train, yTrain,
                      epochs=100,
                      validation_data=(X_test, yTest))
  history_m = {
  "loss": history_m.history["loss"][0],
  "val_loss": history_m.history["val_loss"][0],
  "accuracy": history_m.history["accuracy"][0],
  "val_accuracy": history_m.history["val_accuracy"][0],
  "val_f1_m": history_m.history["val_f1_m"][0]
  }
  return history_m

# Bayesian Optimization

In [ ]:
pip install hpbandster ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 95 kB/s 
     |████████████████████████████████| 4.9 MB 7.8 MB/s 
     |████████████████████████████████| 89 kB 8.5 MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80008 sha256=47a45d81707ed985bc82910e1d705d6de6f4f4f6f46e26ce39f35a4711e3ef20
  Stored in directory: /root/.cache/pip/wheels/d9/88/fc/61ab6b9f386a386839668631c39a6dc3c2fb0ec7000d552faa
Successfully built hpbandster


In [ ]:
pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ConfigSpace as CS
config_space = CS.ConfigurationSpace()

In [ ]:
config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="lr", choices=[ 0.0001, 0.001, 0.01, 0.1]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="dropout_rate", choices=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block1_filters", choices=[8, 16, 32, 64, 128]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc_layer_type", choices= ['dense', 'convolution']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="pool_type", choices= ['max', 'average']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc1_units", choices=[8, 16, 32, 64, 128]))

# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="lr", choices=[0.001]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="dropout_rate", choices=[ 0.3])) 
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="conv_block1_filters", choices=[8]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="fc_layer_type", choices= ['dense']))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="pool_type", choices= [ 'average']))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter(
#         name="fc1_units", choices=[16]))

fc1_units, Type: Categorical, Choices: {8, 16, 32, 64, 128}, Default: 8

In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
algo = TuneBOHB(
    config_space, metric="val_f1_m", mode="max", max_concurrent=1)
bohb = HyperBandForBOHB(
    time_attr="training_iteration",
    metric="val_f1_m",
      mode="max",
      max_t=2)
tune.run(train_mnist, scheduler=bohb, num_samples=3, search_alg=algo)

2022-11-22 08:17:16,673	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(train_mnist pid=840) 2022-11-22 08:17:36.526676: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=840) KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='dense_2/activation_2/Softmax:0', description="created by layer 'dense_2'")
(train_mnist pid=840) Model: "model"
(train_mnist pid=840) _________________________________________________________________
(train_mnist pid=840)  Layer (type)                Output Shape              Param #   
(train_mnist pid=840) =================================================================
(train_mnist pid=840)  input_1 (InputLayer)        [(None, 43893, 1)]        0         
(train_mnist pid=840)                                                                  
(train_mnist pid=840)  conv1d (Conv1D)             (None, 43886, 128)        1152      
(train_mnist pid=840)                                                                  
(train_mnist pid=840)  batch_normalization (BatchN  (None, 43886, 128)       512       
(train_mnist pid=840)  ormalization)                                        

(train_mnist pid=840) 2022-11-22 08:17:36.842260: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 719028224 exceeds 10% of free system memory.
(train_mnist pid=840) 2022-11-22 08:17:37.357316: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 719028224 exceeds 10% of free system memory.
(train_mnist pid=840) 2022-11-22 08:17:37.966872: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 719028224 exceeds 10% of free system memory.
(train_mnist pid=840) 2022-11-22 08:17:38.672719: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 719028224 exceeds 10% of free system memory.
(train_mnist pid=840) 2022-11-22 08:17:39.047881: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 719028224 exceeds 10% of free system memory.
(train_mnist pid=840) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7fb40f3cc050> and will run it as-is.
(train_mnist pid=840) Cause: Unable to locate the source 

5/5 [==============================] - 33s 6s/step - loss: 1.0908 - accuracy: 0.4583 - f1_m: 0.4219 - precision_m: 0.4595 - recall_m: 0.4688 - val_loss: 1.9931 - val_accuracy: 0.1622 - val_f1_m: 0.0544 - val_precision_m: 0.8541 - val_recall_m: 0.1781
(train_mnist pid=840) Epoch 2/100
5/5 [==============================] - 29s 6s/step - loss: 1.0366 - accuracy: 0.4653 - f1_m: 0.2819 - precision_m: 0.6965 - recall_m: 0.4437 - val_loss: 2.4752 - val_accuracy: 0.4865 - val_f1_m: 0.2810 - val_precision_m: 0.7550 - val_recall_m: 0.4500
(train_mnist pid=840) Epoch 3/100
5/5 [==============================] - 33s 6s/step - loss: 1.0442 - accuracy: 0.4375 - f1_m: 0.3120 - precision_m: 0.6405 - recall_m: 0.4500 - val_loss: 1.1452 - val_accuracy: 0.4865 - val_f1_m: 0.2810 - val_precision_m: 0.7550 - val_recall_m: 0.4500
(train_mnist pid=840) Epoch 4/100
5/5 [==============================] - 29s 6s/step - loss: 1.0334 - accuracy: 0.4653 - f1_m: 0.3187 - precision_m: 0.7600 - recall_m: 0.4812 - va

...

(train_mnist pid=4599) 2022-11-22 09:37:40.589472: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=4599) KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='dense_2/activation_2/Softmax:0', description="created by layer 'dense_2'")
(train_mnist pid=4599) Model: "model"
(train_mnist pid=4599) _________________________________________________________________
(train_mnist pid=4599)  Layer (type)                Output Shape              Param #   
(train_mnist pid=4599) =================================================================
(train_mnist pid=4599)  input_1 (InputLayer)        [(None, 43893, 1)]        0         
(train_mnist pid=4599)                                                                  
(train_mnist pid=4599)  conv1d (Conv1D)             (None, 43886, 128)        1152      
(train_mnist pid=4599)                                                                  
(train_mnist pid=4599)  batch_normalization (BatchN  (None, 43886, 128)       512       
(train_mnist pid=4599)  ormalization)                             

(train_mnist pid=4599) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f91271a4050> and will run it as-is.
(train_mnist pid=4599) Cause: Unable to locate the source code of <function f1_m at 0x7f91271a4050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(train_mnist pid=4599) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(train_mnist pid=4599) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7f91271a4050> and will run it as-is.
(train_mnist pid=4599) Cause: Unable to locate the source code of <function f1_m at 0x7f91271a4050>. Note that functions defined in certain environments, like the interactive Python s

5/5 [==============================] - 47s 9s/step - loss: 1.5314 - accuracy: 0.2292 - f1_m: 0.2150 - precision_m: 0.3622 - recall_m: 0.2250 - val_loss: 1.0823 - val_accuracy: 0.4054 - val_f1_m: 0.4558 - val_precision_m: 0.5592 - val_recall_m: 0.4875
(train_mnist pid=4599) Epoch 2/100
5/5 [==============================] - 37s 8s/step - loss: 1.3640 - accuracy: 0.3056 - f1_m: 0.3059 - precision_m: 0.5164 - recall_m: 0.3187 - val_loss: 1.1074 - val_accuracy: 0.1622 - val_f1_m: 0.0296 - val_precision_m: 0.9238 - val_recall_m: 0.0938
(train_mnist pid=4599) Epoch 3/100
5/5 [==============================] - 38s 7s/step - loss: 1.3056 - accuracy: 0.3333 - f1_m: 0.3495 - precision_m: 0.5649 - recall_m: 0.3438 - val_loss: 1.0978 - val_accuracy: 0.4595 - val_f1_m: 0.2531 - val_precision_m: 0.6781 - val_recall_m: 0.3500
(train_mnist pid=4599) Epoch 4/100
5/5 [==============================] - 34s 7s/step - loss: 1.2453 - accuracy: 0.3542 - f1_m: 0.3168 - precision_m: 0.5315 - recall_m: 0.3562 -

2022-11-22 10:40:52,626	INFO tune.py:778 -- Total run time: 8616.00 seconds (8615.80 seconds for the tuning loop).
